In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
def PolyCoefficients(x, coeffs):
    """
    Compute the polynomial value for given 'x' based on provided coefficients.

    Parameters:
    - x (float): The input value at which the polynomial is evaluated.
    - coeffs (list of float): The coefficients of the polynomial, in ascending order of power.

    Returns:
    - float: The polynomial value at 'x'.
    """
    
    o = len(coeffs)
    print(f'# This is a polynomial of order {o}.')
    
    y = 0
    for i in range(o):
        y += coeffs[i]*x**i
        
    return y

def extract_statistical_properties(filepath):
    """
    Extract statistical properties from a CSV file and filter the data based on certain conditions.
    
    Parameters:
    - filepath (str): The path to the CSV file containing the data.
    
    Returns:
    - dict: A dictionary containing mean and standard deviation for specific columns, and the correlation between two columns.
    """
    
    df = pd.read_csv(filepath, encoding='unicode_escape')
    df = df[df['Volum_aproximare_metoda_semiautomata'].notna()]
    df = df[df['Diagnostic'].str.contains("NT") == False]

    stats = {
        'mean_x': df['Volum_aproximare_metoda_semiautomata'].mean(),
        'std_x': df['Volum_aproximare_metoda_semiautomata'].std(),
        'mean_y': df['Volum_metoda_segmentare_normala'].mean(),
        'std_y': df['Volum_metoda_segmentare_normala'].std(),
        'corr': df[['Volum_aproximare_metoda_semiautomata', 'Volum_metoda_segmentare_normala']].corr().iloc[0, 1]
    }

    return stats

def generate_valid_x_y_pair(n, mean_x, std_x, coeffs, noise_level=0.0):
    """
    Generate valid (x, y) pairs with noise added to the y values, ensuring both are positive.

    Parameters:
    - n (int): The number of data points to generate.
    - mean_x (float): The mean of the x values distribution.
    - std_x (float): The standard deviation of the x values distribution.
    - coeffs (list of float): The coefficients of the polynomial.
    - noise_level (float): The standard deviation of the noise added to the y values.

    Returns:
    - tuple: Two numpy arrays containing the generated x and y values.
    """
    
    np.random.seed(42)
    valid_x = []
    valid_y = []

    while len(valid_x) < n:
        x_candidate = np.random.normal(loc=mean_x, scale=std_x, size=1)[0]
        y_candidate = np.polyval(coeffs, x_candidate)
        y_candidate += np.random.normal(0, noise_level, 1)[0]

        if x_candidate > 0 and y_candidate > 0:
            valid_x.append(x_candidate)
            valid_y.append(y_candidate)

    return np.array(valid_x), np.array(valid_y)

def generate_synthetic_data_2nd_degree(n, mean_x, std_x):
    """
    Generate synthetic data based on a 2nd-degree polynomial using the generate_valid_x_y_pair function.

    Parameters:
    - n (int): The number of data points to generate.
    - mean_x (float): The mean of the x values distribution.
    - std_x (float): The standard deviation of the x values distribution.

    Returns:
    - DataFrame: A pandas DataFrame containing the generated synthetic data.
    """
    
    coeffs = [-2.587e-06, 1.3126, -1561.8436] 

    x_values, y_values = generate_valid_x_y_pair(n, mean_x, std_x, coeffs, 8000)

    df_synthetic = pd.DataFrame({
        'Volum_aproximare_metoda_semiautomata': x_values,
        'Volum_metoda_segmentare_normala': y_values
    })

    return df_synthetic

def generate_synthetic_data_3rd_degree(n, mean_x, std_x):
    """
    Generate synthetic data based on a 3rd-degree polynomial using the generate_valid_x_y_pair function.

    Parameters:
    - n (int): The number of data points to generate.
    - mean_x (float): The mean of the x values distribution.
    - std_x (float): The standard deviation of the x values distribution.

    Returns:
    - DataFrame: A pandas DataFrame containing the generated synthetic data.
    """
    
    coeffs = [-2.563e-11, 5.2e-06, 0.8022, 5.49e-05]

    x_values, y_values = generate_valid_x_y_pair(n, mean_x, std_x, coeffs, 8000)

    df_synthetic = pd.DataFrame({
        'Volum_aproximare_metoda_semiautomata': x_values,
        'Volum_metoda_segmentare_normala': y_values
    })

    return df_synthetic

In [ ]:
# We read and filter the CSV file into a DataFrame
df = pd.read_csv('data_tur1.csv', encoding='unicode_escape')
df = df[df['Volum_aproximare_metoda_semiautomata'].notna()]
df = df[df['Diagnostic'].str.contains("NT") == False]

print('Data \n')
print(df.head())
print('\n')

In [ ]:
# We convert volume columns of the DataFrame to numpy arrays and reshape them for modeling
x = np.array(df['Volum_aproximare_metoda_semiautomata'].tolist()).reshape((-1, 1))
y = np.array(df['Volum_metoda_segmentare_normala'].tolist())

print(x.shape)
print(y.shape)

In [ ]:
# We create polynomial features, fit the OLS model, and print model summary and coefficients
polynomial_features = PolynomialFeatures(degree=2) # Degree of the polynomial function

xp = polynomial_features.fit_transform(x)
xp = sm.add_constant(xp)

model = sm.OLS(y, xp).fit()

results = model.predict(xp)
SDE, upper, lower = wls_prediction_std(model, alpha=0.05)

print(model.summary())
print('\n')
print(f"Predicted values:\n{results}")
print('\n')
print("Coefficients: " + str(model.params))

In [ ]:
# We plot the polynomial function and ground truth points
plt.figure(figsize=(32, 18))
plt.title('$2^{nd}$ Degree Polynomial Function (continuous)', fontsize=32, pad=15)
plt.xlabel('Volume (Semi-Automatic Method)', fontsize=32, labelpad=16)
plt.ylabel('Volume Prediction', fontsize=32, labelpad=16)

plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['top'].set_linewidth(2)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_linewidth(2)
plt.gca().spines['right'].set_color('black')

plt.tick_params(axis='both', which='major', labelsize=30, width=2)
plt.grid(True, linestyle='-', linewidth=1, color='gray')
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)

x2 = np.arange(min(x), max(x))
y2 = PolyCoefficients(x2, model.params)

plt.plot(x2, y2, color='green', label='Predicted Function', linewidth=3)
plt.scatter(x, y, color='red', label='Ground Truth Points')
plt.legend(loc='upper left', fontsize=26)

plt.savefig("2nd_degree_prediction_ground_truth_continuous", dpi=500, bbox_inches='tight', transparent=True)

x2, y2 = zip(*sorted(zip(x, results),key=lambda x: x[0]))
x2, lower = zip(*sorted(zip(x, lower),key=lambda x: x[0]))
x2, upper = zip(*sorted(zip(x, upper),key=lambda x: x[0]))

plt.figure(figsize=(32, 18))
plt.title('$2^{nd}$ Degree Polynomial Function (discrete)', fontsize=32, pad=15)
plt.xlabel('Volume (Semi-Automatic Method)', fontsize=32, labelpad=16)
plt.ylabel('Volume Prediction', fontsize=32, labelpad=16)

plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['top'].set_linewidth(2)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_linewidth(2)
plt.gca().spines['right'].set_color('black')

plt.tick_params(axis='both', which='major', labelsize=30, width=2)
plt.grid(True, linestyle='-', linewidth=1, color='gray')
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)

plt.plot(x2, y2, color='green', label='Predicted Function', linewidth=3)
plt.plot(x2, upper, '--', label="95% CI Upper Bound", linewidth=3)
plt.plot(x2, lower, ':', label="95% CI Lower Bound", linewidth=3)
plt.legend(loc='upper left', fontsize=26)

plt.savefig("2nd_degree_prediction_95CI_discrete", dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# We generate synthetic data based on the original data's statistical properties and save it to CSV files
filepath = 'data_tur1.csv'
stats = extract_statistical_properties(filepath)

df_synthetic_2nd_degree = generate_synthetic_data_2nd_degree(93, stats['mean_x'], stats['std_x'])
df_synthetic_3rd_degree = generate_synthetic_data_3rd_degree(93, stats['mean_x'], stats['std_x']) 

df_synthetic_2nd_degree.to_csv('T_synthetic_data_2nd_degree.csv', index=False)
df_synthetic_3rd_degree.to_csv('T_synthetic_data_3rd_degree.csv', index=False)

print('Enhanced synthetic data based on 2nd and 3rd-degree polynomials generated and saved successfully!')

In [ ]:
# We combine the synthetic and original data, fit a polynomial model, and plot the results
df_synthetic = pd.read_csv('T_synthetic_data_3rd_degree.csv', encoding='unicode_escape')
df_synthetic = df_synthetic[df_synthetic['Volum_aproximare_metoda_semiautomata'].notna()]

df_original = pd.read_csv('data_tur1.csv', encoding='unicode_escape')
df_original = df_original[df_original['Volum_aproximare_metoda_semiautomata'].notna()]
df_original = df_original[df_original['Diagnostic'].str.contains("NT") == False]

df = pd.concat([df_synthetic, df_original], ignore_index=True)

x = np.array(df['Volum_aproximare_metoda_semiautomata'].tolist()).reshape((-1, 1))
y = np.array(df['Volum_metoda_segmentare_normala'].tolist())

polynomial_features = PolynomialFeatures(degree=2) # Degree of the polynomial function

xp = polynomial_features.fit_transform(x)
xp = sm.add_constant(xp)

model = sm.OLS(y, xp).fit()

results = model.predict(xp)
SDE, upper, lower = wls_prediction_std(model, alpha=0.05)

print(model.summary())
print('\n')
print(f"Predicted values:\n{results}")
print('\n')
print("Coefficients: " + str(model.params))

plt.figure(figsize=(32, 18))
plt.title('$2^{nd}$ Degree Polynomial Function (continuous)', fontsize=32, pad=15)
plt.xlabel('Volume (Semi-Automatic Method)', fontsize=32, labelpad=16)
plt.ylabel('Volume Prediction', fontsize=32, labelpad=16)

plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['top'].set_linewidth(2)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_linewidth(2)
plt.gca().spines['right'].set_color('black')

plt.tick_params(axis='both', which='major', labelsize=30, width=2)
plt.grid(True, linestyle='-', linewidth=1, color='gray')
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)

x2 = np.arange(min(x), max(x))
y2 = PolyCoefficients(x2, model.params)

plt.plot(x2, y2, color='green', label='Predicted Function', linewidth=3)
plt.scatter(x, y, color='red', label='Ground Truth Points')
plt.legend(loc='upper left', fontsize=26)

plt.savefig("2nd_degree_prediction_ground_truth_continuous_synthetic", dpi=500, bbox_inches='tight', transparent=True)

x2, y2 = zip(*sorted(zip(x, results),key=lambda x: x[0]))
x2, lower = zip(*sorted(zip(x, lower),key=lambda x: x[0]))
x2, upper = zip(*sorted(zip(x, upper),key=lambda x: x[0]))

plt.figure(figsize=(32, 18))
plt.title('$2^{nd}$ Degree Polynomial Function (discrete)', fontsize=32, pad=15)
plt.xlabel('Volume (Semi-Automatic Method)', fontsize=32, labelpad=16)
plt.ylabel('Volume Prediction', fontsize=32, labelpad=16)

plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['top'].set_linewidth(2)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_linewidth(2)
plt.gca().spines['right'].set_color('black')

plt.tick_params(axis='both', which='major', labelsize=30, width=2)
plt.grid(True, linestyle='-', linewidth=1, color='gray')
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)

plt.plot(x2, y2, color='green', label='Predicted Function', linewidth=3)
plt.plot(x2, upper, '--', label="95% CI Upper Bound", linewidth=3)
plt.plot(x2, lower, ':', label="95% CI Lower Bound", linewidth=3)
plt.legend(loc='upper left', fontsize=26)

plt.savefig("2nd_degree_prediction_95CI_discrete_synthetic", dpi=300, bbox_inches='tight', transparent=True)